In [1]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv('qualityOflife.csv')

print("原始数据 - 第1行的KEPAQ值：")
print(f"KEPAQ_F_Q01: {df.loc[0, 'KEPAQ_F_Q01']}")
print(f"KEPAQ_F_Q02: {df.loc[0, 'KEPAQ_F_Q02']}")
print(f"KEPAQ_F_Q03: {df.loc[0, 'KEPAQ_F_Q03']}")

# 删除NAME和ID列，添加新的PATIENT_ID
df = df.drop(['NAME', 'ID'], axis=1)
df.insert(0, 'PATIENT_ID', range(1, len(df) + 1))

# 定义KEPAQ问卷的item列（保持这个顺序）
kepaq_items = [
    'KEPAQ_F_Q01', 'KEPAQ_F_Q02', 'KEPAQ_F_Q03', 'KEPAQ_F_Q04', 
    'KEPAQ_F_Q05', 'KEPAQ_F_Q06', 'KEPAQ_F_Q07', 'KEPAQ_F_Q08', 
    'KEPAQ_F_Q09', 'KEPAQ_E_Q01', 'KEPAQ_E_Q02', 'KEPAQ_E_Q03', 
    'KEPAQ_E_Q04', 'KEPAQ_E_Q05', 'KEPAQ_E_Q06', 'KEPAQ_E_Q07'
]

# 定义协变量列
covariate_cols = [
    'AGE', 'SEX', 'AGE_AT_DIAGNOSIS', 'AGE_AT_FIRST_SURGERY',
    'REFRACTIVE_SITUATION', 'KERATOPLASTY', 'CORNEAL_RINGS', 
    'CROSSLINKING', 'INTRAOCULAR_LENS', 'HYPERTENSION', 'DIABETES',
    'HYPERTIROIDISM', 'HIPOTHYROIDISM', 'ALLERGIC_CONJUNCTIVITIS',
    'DERMATITIS', 'FOOD_ALERGY', 'DRUGS_ALERGY', 'KIDNEY_DISEASE',
    'LIVER_DISEASE', 'MARFAN', 'EHLERS-DAHNLOS', 'RIEGER', 'DOWN',
    'APERT', 'OCULODENTODIGITAL', 'TURNER'
]

# 转换为长格式
df_long = pd.melt(
    df,
    id_vars=['PATIENT_ID'] + covariate_cols,
    value_vars=kepaq_items,
    var_name='item',
    value_name='resp'
)

# 重命名列
df_long = df_long.rename(columns={'PATIENT_ID': 'id'})

# 为协变量添加cov_前缀
covariate_rename = {col: f'cov_{col.lower()}' for col in covariate_cols}
df_long = df_long.rename(columns=covariate_rename)

# 创建item的顺序映射，保持原始定义的顺序
item_order = {item: i for i, item in enumerate(kepaq_items)}
df_long['item_order'] = df_long['item'].map(item_order)

# 按id和item_order排序（保持item的原始顺序）
df_long = df_long.sort_values(['id', 'item_order']).reset_index(drop=True)

# 删除临时的item_order列
df_long = df_long.drop('item_order', axis=1)

# 重新排列列的顺序
cov_columns = [f'cov_{col.lower()}' for col in covariate_cols]
df_long = df_long[['id', 'item', 'resp'] + cov_columns]

# 验证转换后的数据
print("\n转换后 - 患者1的前5个item：")
patient_1_data = df_long[df_long['id'] == 1].head(5)
print(patient_1_data[['id', 'item', 'resp']])

# 保存文件
df_long.to_csv('qualityOflife_long_format.csv', index=False, encoding='utf-8')

print(f"\n✓ 转换完成！共 {len(df_long)} 行")

原始数据 - 第1行的KEPAQ值：
KEPAQ_F_Q01: 0.0
KEPAQ_F_Q02: 2.0
KEPAQ_F_Q03: 1.0

转换后 - 患者1的前5个item：
   id         item  resp
0   1  KEPAQ_F_Q01   0.0
1   1  KEPAQ_F_Q02   2.0
2   1  KEPAQ_F_Q03   1.0
3   1  KEPAQ_F_Q04   2.0
4   1  KEPAQ_F_Q05   1.0

✓ 转换完成！共 6176 行
